In [27]:
# 导包
import talib
import pandas as pd
import akshare as ak
from datetime import datetime
import os

In [ ]:
# 模块化封装
# 注意 为了保证封装的数据普适性，数据有任意一个缺失的行的都选择了全部删除。为追求准确性，比如只分析 close 和 date（如动量因子），保留 close 就足够（用的时候看可以加上）
#######################
"""
def load_futures_data(filepath: str) -> pd.DataFrame:
    """
    加载国债期货 CSV 数据，统一处理日期格式，仅保留 symbol、date、close 不为空的行，
    其余字段允许缺失。返回包含：
    symbol, date, open, high, low, close, volume, open_interest, turnover, settle, pre_settle
    """
    df = pd.read_csv(filepath)
    df["date"] = pd.to_datetime(df["date"], format="%Y%m%d", errors="coerce")
    cols = ["symbol", "date", "open", "high", "low", "close", "volume", "open_interest", "turnover", "settle", "pre_settle"]
    
    # 只丢弃 symbol/date/close 缺失的行，保留其他缺失
    return df[cols].dropna(subset=["symbol", "date", "close"])
"""
####################################

import pandas as pd

def load_futures_data(filepath: str) -> pd.DataFrame:
    """
    加载国债期货 CSV 数据，统一处理日期格式，保留所有分析所需字段。
    返回包含：symbol, date, open, high, low, close, volume, open_interest, turnover, settle, pre_settle
    """
    df = pd.read_csv(filepath)
    df["date"] = pd.to_datetime(df["date"], format="%Y%m%d", errors="coerce")
    cols = ["symbol", "date", "open", "high", "low", "close", "volume", "open_interest", "turnover", "settle", "pre_settle"]
    return df[cols].dropna()

# 调用数据
df = load_futures_data(
    r"C:\Users\29947\Desktop\浙商证券 固收转债+量化\利率择时量化项目\国债数据\T_10Y_Bond_Futures.csv"
)
# 导出为 Excel 文件查看
df.to_excel(r"C:\Users\29947\Desktop\T_10Y_Bond_Futures_preview.xlsx", index=False)

# TA-LIB Momentum Indicator Functions（动量）

In [39]:
# =============ADX 指标（Average Directional Index）=================#
#   - ADX：趋势强度
#   - +DI：正向动能（多头）
#   - -DI：负向动能（空头）


# 计算 ADX
results = []
for symbol in df["symbol"].unique():
    df_sym = df[df["symbol"] == symbol].copy()
    df_sym.sort_values("date", inplace=True)

    df_sym["adx"] = talib.ADX(
        df_sym["high"].values,
        df_sym["low"].values,
        df_sym["close"].values,
        timeperiod=14
    )

    results.append(df_sym[["symbol", "date", "adx"]])

ADX = pd.concat(results, ignore_index=True)
ADX["date"] = ADX["date"].dt.strftime("%Y/%#m/%#d")
ADX

,symbol,date,adx
0,T1509,2015/3/20,NaN
1,T1509,2015/3/23,NaN
2,T1509,2015/3/24,NaN
3,T1509,2015/3/25,NaN
4,T1509,2015/3/26,NaN
...,...,...,...
7334,T2512,2025/5/13,49.765517
7335,T2512,2025/5/14,47.861618
7336,T2512,2025/5/15,46.274279
7337,T2512,2025/5/16,45.216327


In [ ]:
# ADXR
results = []
for symbol in df["symbol"].unique():
    d = df[df["symbol"] == symbol].sort_values("date").copy()
    d["ADXR"] = talib.ADXR(d["high"], d["low"], d["close"], timeperiod=14)
    results.append(d[["symbol", "date", "ADXR"]])

ADXR = pd.concat(results, ignore_index=True)
ADXR["date"] = ADXR ["date"].dt.strftime("%Y/%#m/%#d")
ADXR.adxr.to_excel(r"C:\Users\29947\Desktop\ADXR.xlsx", index=False)

In [ ]:
# 计算APO：用于趋势判断，APO > 0 视为偏多，APO < 0 偏空

results = []

for symbol in df["symbol"].unique():
    d = df[df["symbol"] == symbol].sort_values("date").copy()
    d["APO"] = talib.APO(d["close"], fastperiod=12, slowperiod=26, matype=0)
    results.append(d[["symbol", "date", "APO"]])

apo = pd.concat(results, ignore_index=True)
apo["date"] = apo["date"].dt.strftime("%Y/%#m/%#d")

print(apo)

     symbol       date       APO
0     T1509  2015/3/20       NaN
1     T1509  2015/3/23       NaN
2     T1509  2015/3/24       NaN
3     T1509  2015/3/25       NaN
4     T1509  2015/3/26       NaN
...     ...        ...       ...
7334  T2512  2025/5/13  0.038045
7335  T2512  2025/5/14 -0.006603
7336  T2512  2025/5/15 -0.027212
7337  T2512  2025/5/16 -0.030224
7338  T2512  2025/5/19 -0.042468

[7339 rows x 3 columns]


In [ ]:
# AROON（阿隆指标），用于衡量价格是否接近近期高/低点，判断趋势强度与启动时机
# 计算 AROONUP（接近最高点的程度）与 AROONDOWN（接近最低点的程度）

import array
results = []
for symbol in df["symbol"].unique():
    d = df[df["symbol"] == symbol].sort_values("date").copy()
    d["AROONUP"], d["AROONDOWN"] = talib.AROON(d["high"], d["low"], timeperiod=14)
    results.append(d[["symbol", "date", "AROONUP", "AROONDOWN"]])

aroon = pd.concat(results, ignore_index=True)
aroon["date"] = aroon["date"].dt.strftime("%Y/%#m/%#d")
aroon

,symbol,date,AROONUP,AROONDOWN
0,T1509,2015/3/20,NaN,NaN
1,T1509,2015/3/23,NaN,NaN
2,T1509,2015/3/24,NaN,NaN
3,T1509,2015/3/25,NaN,NaN
4,T1509,2015/3/26,NaN,NaN
...,...,...,...,...
7334,T2512,2025/5/13,92.857143,85.714286
7335,T2512,2025/5/14,85.714286,78.571429
7336,T2512,2025/5/15,78.571429,71.428571
7337,T2512,2025/5/16,100.000000,64.285714


In [35]:
# AROONOSC（阿隆振荡器），表示 AROONUP 与 AROONDOWN 的差值，范围 [-100, 100]
# 趋势信号指标，正值表示上涨趋势占优，负值表示下跌趋势占优

results = []

for symbol in df["symbol"].unique():
    d = df[df["symbol"] == symbol].sort_values("date").copy()
    d["AROONOSC"] = talib.AROONOSC(d["high"], d["low"], timeperiod=14)
    results.append(d[["symbol", "date", "AROONOSC"]])

aroonosc = pd.concat(results, ignore_index=True)
aroonosc["date"] = aroonosc["date"].dt.strftime("%Y/%#m/%#d")
aroonosc

,symbol,date,AROONOSC
0,T1509,2015/3/20,NaN
1,T1509,2015/3/23,NaN
2,T1509,2015/3/24,NaN
3,T1509,2015/3/25,NaN
4,T1509,2015/3/26,NaN
...,...,...,...
7334,T2512,2025/5/13,-7.142857
7335,T2512,2025/5/14,-7.142857
7336,T2512,2025/5/15,-7.142857
7337,T2512,2025/5/16,-35.714286


In [40]:
# BOP（Balance of Power，力量平衡指标），衡量多空双方在一段时间内的博弈强度

results = []

for symbol in df["symbol"].unique():
    d = df[df["symbol"] == symbol].sort_values("date").copy()
    d["BOP"] = talib.BOP(d["open"], d["high"], d["low"], d["close"])
    results.append(d[["symbol", "date", "BOP"]])

bop = pd.concat(results, ignore_index=True)
bop["date"] = bop["date"].dt.strftime("%Y/%#m/%#d")
bop

,symbol,date,BOP
0,T1509,2015/3/20,0.132404
1,T1509,2015/3/23,-0.294118
2,T1509,2015/3/24,-0.623188
3,T1509,2015/3/25,-0.629630
4,T1509,2015/3/26,-0.894231
...,...,...,...
7334,T2512,2025/5/13,0.720000
7335,T2512,2025/5/14,-0.695652
7336,T2512,2025/5/15,-0.137931
7337,T2512,2025/5/16,0.348837


In [ ]:
# CCI（商品通道指数），用于衡量价格相对于其统计均值的偏离程度，常用于超买超卖判断 越小越有反弹机会

results = []

for symbol in df["symbol"].unique():
    d = df[df["symbol"] == symbol].sort_values("date").copy()
    d["CCI"] = talib.CCI(d["high"], d["low"], d["close"], timeperiod=14)
    results.append(d[["symbol", "date", "CCI"]])

cci = pd.concat(results, ignore_index=True)
cci["date"] = cci["date"].dt.strftime("%Y/%#m/%#d")
cci

,symbol,date,CCI
0,T1509,2015/3/20,NaN
1,T1509,2015/3/23,NaN
2,T1509,2015/3/24,NaN
3,T1509,2015/3/25,NaN
4,T1509,2015/3/26,NaN
...,...,...,...
7334,T2512,2025/5/13,-112.629400
7335,T2512,2025/5/14,-143.946057
7336,T2512,2025/5/15,-142.285413
7337,T2512,2025/5/16,-140.943267


In [43]:
# CMO（Chande 动量振荡器），衡量价格动量强度，类似 RSI，但对涨跌对称处理，适用于震荡与反转信号判断

results = []

for symbol in df["symbol"].unique():
    d = df[df["symbol"] == symbol].sort_values("date").copy()
    d["CMO"] = talib.CMO(d["close"], timeperiod=14)
    results.append(d[["symbol", "date", "CMO"]])

cmo = pd.concat(results, ignore_index=True)
cmo["date"] = cmo["date"].dt.strftime("%Y/%#m/%#d")
cmo

,symbol,date,CMO
0,T1509,2015/3/20,NaN
1,T1509,2015/3/23,NaN
2,T1509,2015/3/24,NaN
3,T1509,2015/3/25,NaN
4,T1509,2015/3/26,NaN
...,...,...,...
7334,T2512,2025/5/13,7.794651
7335,T2512,2025/5/14,-2.272055
7336,T2512,2025/5/15,-2.015559
7337,T2512,2025/5/16,-4.087952


In [44]:
# DX（方向性指数）：衡量 +DI 与 -DI 的差异程度，反映趋势强度，值越高趋势越明确

results = []

for symbol in df["symbol"].unique():
    d = df[df["symbol"] == symbol].sort_values("date").copy()
    d["DX"] = talib.DX(d["high"], d["low"], d["close"], timeperiod=14)
    results.append(d[["symbol", "date", "DX"]])

dx = pd.concat(results, ignore_index=True)
dx["date"] = dx["date"].dt.strftime("%Y/%#m/%#d")
dx

,symbol,date,DX
0,T1509,2015/3/20,NaN
1,T1509,2015/3/23,NaN
2,T1509,2015/3/24,NaN
3,T1509,2015/3/25,NaN
4,T1509,2015/3/26,NaN
...,...,...,...
7334,T2512,2025/5/13,16.235985
7335,T2512,2025/5/14,23.110930
7336,T2512,2025/5/15,25.638882
7337,T2512,2025/5/16,31.462941


In [45]:
# MACD（指数平滑异同移动平均线）：用于判断趋势方向与动量转折，macd 为快慢均线差

results = []

for symbol in df["symbol"].unique():
    d = df[df["symbol"] == symbol].sort_values("date").copy()
    d["MACD"], d["MACDSIGNAL"], d["MACDHIST"] = talib.MACD(
        d["close"], fastperiod=12, slowperiod=26, signalperiod=9
    )
    results.append(d[["symbol", "date", "MACD", "MACDSIGNAL", "MACDHIST"]])

macd = pd.concat(results, ignore_index=True)
macd["date"] = macd["date"].dt.strftime("%Y/%#m/%#d")
macd

,symbol,date,MACD,MACDSIGNAL,MACDHIST
0,T1509,2015/3/20,NaN,NaN,NaN
1,T1509,2015/3/23,NaN,NaN,NaN
2,T1509,2015/3/24,NaN,NaN,NaN
3,T1509,2015/3/25,NaN,NaN,NaN
4,T1509,2015/3/26,NaN,NaN,NaN
...,...,...,...,...,...
7334,T2512,2025/5/13,0.271602,0.407324,-0.135722
7335,T2512,2025/5/14,0.225209,0.370901,-0.145692
7336,T2512,2025/5/15,0.186693,0.334059,-0.147366
7337,T2512,2025/5/16,0.151198,0.297487,-0.146289


In [46]:
# MACDEXT：可自定义移动平均类型的 MACD 变体，适用于调整平滑方式以匹配不同市场风格

results = []

for symbol in df["symbol"].unique():
    d = df[df["symbol"] == symbol].sort_values("date").copy()
    d["MACD"], d["MACDSIGNAL"], d["MACDHIST"] = talib.MACDEXT(
        d["close"],
        fastperiod=12, fastmatype=0,
        slowperiod=26, slowmatype=0,
        signalperiod=9, signalmatype=0
    )
    results.append(d[["symbol", "date", "MACD", "MACDSIGNAL", "MACDHIST"]])

macdext = pd.concat(results, ignore_index=True)
macdext["date"] = macdext["date"].dt.strftime("%Y/%#m/%#d")
macdext

,symbol,date,MACD,MACDSIGNAL,MACDHIST
0,T1509,2015/3/20,NaN,NaN,NaN
1,T1509,2015/3/23,NaN,NaN,NaN
2,T1509,2015/3/24,NaN,NaN,NaN
3,T1509,2015/3/25,NaN,NaN,NaN
4,T1509,2015/3/26,NaN,NaN,NaN
...,...,...,...,...,...
7334,T2512,2025/5/13,0.038045,0.247158,-0.209113
7335,T2512,2025/5/14,-0.006603,0.197265,-0.203868
7336,T2512,2025/5/15,-0.027212,0.150075,-0.177286
7337,T2512,2025/5/16,-0.030224,0.108743,-0.138967


In [47]:
# MACDFIX：固定周期（12, 26）EMA 的 MACD 版本，仅可调整信号线周期，适合标准快速判断动量变化

results = []

for symbol in df["symbol"].unique():
    d = df[df["symbol"] == symbol].sort_values("date").copy()
    d["MACD"], d["MACDSIGNAL"], d["MACDHIST"] = talib.MACDFIX(
        d["close"], signalperiod=9
    )
    results.append(d[["symbol", "date", "MACD", "MACDSIGNAL", "MACDHIST"]])

macdfix = pd.concat(results, ignore_index=True)
macdfix["date"] = macdfix["date"].dt.strftime("%Y/%#m/%#d")
macdfix

,symbol,date,MACD,MACDSIGNAL,MACDHIST
0,T1509,2015/3/20,NaN,NaN,NaN
1,T1509,2015/3/23,NaN,NaN,NaN
2,T1509,2015/3/24,NaN,NaN,NaN
3,T1509,2015/3/25,NaN,NaN,NaN
4,T1509,2015/3/26,NaN,NaN,NaN
...,...,...,...,...,...
7334,T2512,2025/5/13,0.268607,0.403831,-0.135224
7335,T2512,2025/5/14,0.223713,0.367807,-0.144094
7336,T2512,2025/5/15,0.186273,0.331500,-0.145227
7337,T2512,2025/5/16,0.151740,0.295548,-0.143808


In [ ]:
# MFI（资金流量指标）：结合价格与成交量，衡量资金流入流出强度，类似 RSI，用于识别超买超卖区间.MFI越小，表示资金流出强度越大，价格越超卖，可能预示价格反弹机会。

results = []

for symbol in df["symbol"].unique():
    d = df[df["symbol"] == symbol].sort_values("date").copy()
    d["MFI"] = talib.MFI(d["high"], d["low"], d["close"], d["volume"], timeperiod=14)
    results.append(d[["symbol", "date", "MFI"]])

mfi = pd.concat(results, ignore_index=True)
mfi["date"] = mfi["date"].dt.strftime("%Y/%#m/%#d")
mfi

,symbol,date,MFI
0,T1509,2015/3/20,NaN
1,T1509,2015/3/23,NaN
2,T1509,2015/3/24,NaN
3,T1509,2015/3/25,NaN
4,T1509,2015/3/26,NaN
...,...,...,...
7334,T2512,2025/5/13,54.895294
7335,T2512,2025/5/14,49.199609
7336,T2512,2025/5/15,36.965553
7337,T2512,2025/5/16,32.013990


In [50]:
# MINUS_DI（负方向指标）：衡量下跌动量强度，用于识别趋势方向与转折信号

results = []

for symbol in df["symbol"].unique():
    d = df[df["symbol"] == symbol].sort_values("date").copy()
    d["MINUS_DI"] = talib.MINUS_DI(d["high"], d["low"], d["close"], timeperiod=14)
    results.append(d[["symbol", "date", "MINUS_DI"]])

minus_di = pd.concat(results, ignore_index=True)
minus_di["date"] = minus_di["date"].dt.strftime("%Y/%#m/%#d")
minus_di

,symbol,date,MINUS_DI
0,T1509,2015/3/20,NaN
1,T1509,2015/3/23,NaN
2,T1509,2015/3/24,NaN
3,T1509,2015/3/25,NaN
4,T1509,2015/3/26,NaN
...,...,...,...
7334,T2512,2025/5/13,25.863397
7335,T2512,2025/5/14,27.721215
7336,T2512,2025/5/15,28.008654
7337,T2512,2025/5/16,29.688379


In [51]:
# MINUS_DM（负方向变动值）：衡量当前最低价相对前一最低价的下跌幅度，用于计算趋势动量

results = []

for symbol in df["symbol"].unique():
    d = df[df["symbol"] == symbol].sort_values("date").copy()
    d["MINUS_DM"] = talib.MINUS_DM(d["high"], d["low"], timeperiod=14)
    results.append(d[["symbol", "date", "MINUS_DM"]])

minus_dm = pd.concat(results, ignore_index=True)
minus_dm["date"] = minus_dm["date"].dt.strftime("%Y/%#m/%#d")
minus_dm


,symbol,date,MINUS_DM
0,T1509,2015/3/20,NaN
1,T1509,2015/3/23,NaN
2,T1509,2015/3/24,NaN
3,T1509,2015/3/25,NaN
4,T1509,2015/3/26,NaN
...,...,...,...
7334,T2512,2025/5/13,0.909985
7335,T2512,2025/5/14,0.974986
7336,T2512,2025/5/15,0.955344
7337,T2512,2025/5/16,1.007105


In [52]:
# MOM（动量指标）：衡量当前价格与若干期前价格的差值，反映价格变化速率

results = []

for symbol in df["symbol"].unique():
    d = df[df["symbol"] == symbol].sort_values("date").copy()
    d["MOM"] = talib.MOM(d["close"], timeperiod=10)
    results.append(d[["symbol", "date", "MOM"]])

mom = pd.concat(results, ignore_index=True)
mom["date"] = mom["date"].dt.strftime("%Y/%#m/%#d")
mom

,symbol,date,MOM
0,T1509,2015/3/20,NaN
1,T1509,2015/3/23,NaN
2,T1509,2015/3/24,NaN
3,T1509,2015/3/25,NaN
4,T1509,2015/3/26,NaN
...,...,...,...
7334,T2512,2025/5/13,0.010
7335,T2512,2025/5/14,-0.225
7336,T2512,2025/5/15,-0.310
7337,T2512,2025/5/16,-0.555


In [53]:
# PLUS_DI（正方向指标）：衡量上涨动量强度，用于趋势方向判断

results = []

for symbol in df["symbol"].unique():
    d = df[df["symbol"] == symbol].sort_values("date").copy()
    d["PLUS_DI"] = talib.PLUS_DI(d["high"], d["low"], d["close"], timeperiod=14)
    results.append(d[["symbol", "date", "PLUS_DI"]])

plus_di = pd.concat(results, ignore_index=True)
plus_di["date"] = plus_di["date"].dt.strftime("%Y/%#m/%#d")
plus_di

,symbol,date,PLUS_DI
0,T1509,2015/3/20,NaN
1,T1509,2015/3/23,NaN
2,T1509,2015/3/24,NaN
3,T1509,2015/3/25,NaN
4,T1509,2015/3/26,NaN
...,...,...,...
7334,T2512,2025/5/13,18.638135
7335,T2512,2025/5/14,17.313316
7336,T2512,2025/5/15,16.577311
7337,T2512,2025/5/16,15.477778


In [54]:
# PLUS_DM（正方向变动值）：衡量当前最高价相对前一最高价的上涨幅度，用于判断上涨动量

results = []

for symbol in df["symbol"].unique():
    d = df[df["symbol"] == symbol].sort_values("date").copy()
    d["PLUS_DM"] = talib.PLUS_DM(d["high"], d["low"], timeperiod=14)
    results.append(d[["symbol", "date", "PLUS_DM"]])

plus_dm = pd.concat(results, ignore_index=True)
plus_dm["date"] = plus_dm["date"].dt.strftime("%Y/%#m/%#d")
plus_dm

,symbol,date,PLUS_DM
0,T1509,2015/3/20,NaN
1,T1509,2015/3/23,NaN
2,T1509,2015/3/24,NaN
3,T1509,2015/3/25,NaN
4,T1509,2015/3/26,NaN
...,...,...,...
7334,T2512,2025/5/13,0.655769
7335,T2512,2025/5/14,0.608928
7336,T2512,2025/5/15,0.565434
7337,T2512,2025/5/16,0.525045


In [55]:
# PPO（百分比价格振荡器）：衡量快慢均线差值相对于慢均线的百分比，适用于不同价格尺度的动量比较

results = []

for symbol in df["symbol"].unique():
    d = df[df["symbol"] == symbol].sort_values("date").copy()
    d["PPO"] = talib.PPO(d["close"], fastperiod=12, slowperiod=26, matype=0)
    results.append(d[["symbol", "date", "PPO"]])

ppo = pd.concat(results, ignore_index=True)
ppo["date"] = ppo["date"].dt.strftime("%Y/%#m/%#d")
ppo


,symbol,date,PPO
0,T1509,2015/3/20,NaN
1,T1509,2015/3/23,NaN
2,T1509,2015/3/24,NaN
3,T1509,2015/3/25,NaN
4,T1509,2015/3/26,NaN
...,...,...,...
7334,T2512,2025/5/13,0.034926
7335,T2512,2025/5/14,-0.006060
7336,T2512,2025/5/15,-0.024974
7337,T2512,2025/5/16,-0.027744


In [56]:
# ROC（变动率指标）：衡量当前价格相对于若干期前价格的百分比变化，反映市场加速或减速

results = []

for symbol in df["symbol"].unique():
    d = df[df["symbol"] == symbol].sort_values("date").copy()
    d["ROC"] = talib.ROC(d["close"], timeperiod=10)
    results.append(d[["symbol", "date", "ROC"]])

roc = pd.concat(results, ignore_index=True)
roc["date"] = roc["date"].dt.strftime("%Y/%#m/%#d")
roc

,symbol,date,ROC
0,T1509,2015/3/20,NaN
1,T1509,2015/3/23,NaN
2,T1509,2015/3/24,NaN
3,T1509,2015/3/25,NaN
4,T1509,2015/3/26,NaN
...,...,...,...
7334,T2512,2025/5/13,0.009189
7335,T2512,2025/5/14,-0.206697
7336,T2512,2025/5/15,-0.284547
7337,T2512,2025/5/16,-0.508475


In [ ]:
# ROCP（变动率百分比）：计算当前价格相对前期价格的涨跌百分比，反映短期趋势斜率
results = []

for symbol in df["symbol"].unique():
    d = df[df["symbol"] == symbol].sort_values("date").copy()
    d["ROCP"] = talib.ROCP(d["close"], timeperiod=10)
    results.append(d[["symbol", "date", "ROCP"]])

rocp = pd.concat(results, ignore_index=True)
rocp["date"] = rocp["date"].dt.strftime("%Y/%#m/%#d")
rocp

,symbol,date,ROCP
0,T1509,2015/3/20,NaN
1,T1509,2015/3/23,NaN
2,T1509,2015/3/24,NaN
3,T1509,2015/3/25,NaN
4,T1509,2015/3/26,NaN
...,...,...,...
7334,T2512,2025/5/13,0.000092
7335,T2512,2025/5/14,-0.002067
7336,T2512,2025/5/15,-0.002845
7337,T2512,2025/5/16,-0.005085


In [58]:
# ROCR100（变动率比值百分化）：计算当前价格与前期价格的比率并乘以100，结果更直观，基准为100

results = []

for symbol in df["symbol"].unique():
    d = df[df["symbol"] == symbol].sort_values("date").copy()
    d["ROCR100"] = talib.ROCR100(d["close"], timeperiod=10)
    results.append(d[["symbol", "date", "ROCR100"]])

rocr100 = pd.concat(results, ignore_index=True)
rocr100["date"] = rocr100["date"].dt.strftime("%Y/%#m/%#d")
rocr100

,symbol,date,ROCR100
0,T1509,2015/3/20,NaN
1,T1509,2015/3/23,NaN
2,T1509,2015/3/24,NaN
3,T1509,2015/3/25,NaN
4,T1509,2015/3/26,NaN
...,...,...,...
7334,T2512,2025/5/13,100.009189
7335,T2512,2025/5/14,99.793303
7336,T2512,2025/5/15,99.715453
7337,T2512,2025/5/16,99.491525


In [59]:
# RSI（相对强弱指数）：衡量价格上涨与下跌动能的强度比例
results = []

for symbol in df["symbol"].unique():
    d = df[df["symbol"] == symbol].sort_values("date").copy()
    d["RSI"] = talib.RSI(d["close"], timeperiod=14)
    results.append(d[["symbol", "date", "RSI"]])

rsi = pd.concat(results, ignore_index=True)
rsi["date"] = rsi["date"].dt.strftime("%Y/%#m/%#d")
rsi

,symbol,date,RSI
0,T1509,2015/3/20,NaN
1,T1509,2015/3/23,NaN
2,T1509,2015/3/24,NaN
3,T1509,2015/3/25,NaN
4,T1509,2015/3/26,NaN
...,...,...,...
7334,T2512,2025/5/13,53.897325
7335,T2512,2025/5/14,48.863972
7336,T2512,2025/5/15,48.992221
7337,T2512,2025/5/16,47.956024


In [60]:
# STOCH（随机指标）：用于衡量当前价格在近期高低区间的位置，适用于超买超卖判断

results = []

for symbol in df["symbol"].unique():
    d = df[df["symbol"] == symbol].sort_values("date").copy()
    d["SLOWK"], d["SLOWD"] = talib.STOCH(
        d["high"], d["low"], d["close"],
        fastk_period=5,
        slowk_period=3, slowk_matype=0,
        slowd_period=3, slowd_matype=0
    )
    results.append(d[["symbol", "date", "SLOWK", "SLOWD"]])

stoch = pd.concat(results, ignore_index=True)
stoch["date"] = stoch["date"].dt.strftime("%Y/%#m/%#d")
stoch

,symbol,date,SLOWK,SLOWD
0,T1509,2015/3/20,NaN,NaN
1,T1509,2015/3/23,NaN,NaN
2,T1509,2015/3/24,NaN,NaN
3,T1509,2015/3/25,NaN,NaN
4,T1509,2015/3/26,NaN,NaN
...,...,...,...,...
7334,T2512,2025/5/13,45.658171,52.801878
7335,T2512,2025/5/14,29.140461,43.312113
7336,T2512,2025/5/15,27.463312,34.087315
7337,T2512,2025/5/16,20.902295,25.835356


In [61]:
# STOCHF（快速随机指标）：与 STOCH 相似，但波动更敏感，适合短线超买超卖判断

results = []

for symbol in df["symbol"].unique():
    d = df[df["symbol"] == symbol].sort_values("date").copy()
    d["FASTK"], d["FASTD"] = talib.STOCHF(
        d["high"], d["low"], d["close"],
        fastk_period=5,
        fastd_period=3, fastd_matype=0
    )
    results.append(d[["symbol", "date", "FASTK", "FASTD"]])

stochf = pd.concat(results, ignore_index=True)
stochf["date"] = stochf["date"].dt.strftime("%Y/%#m/%#d")
stochf

,symbol,date,FASTK,FASTD
0,T1509,2015/3/20,NaN,NaN
1,T1509,2015/3/23,NaN,NaN
2,T1509,2015/3/24,NaN,NaN
3,T1509,2015/3/25,NaN,NaN
4,T1509,2015/3/26,NaN,NaN
...,...,...,...,...
7334,T2512,2025/5/13,44.025157,45.658171
7335,T2512,2025/5/14,18.867925,29.140461
7336,T2512,2025/5/15,19.496855,27.463312
7337,T2512,2025/5/16,24.342105,20.902295


In [62]:
# STOCHRSI（随机 RSI）：对 RSI 再进行随机指标处理，提升对超买超卖状态的灵敏度，适用于震荡市场

results = []

for symbol in df["symbol"].unique():
    d = df[df["symbol"] == symbol].sort_values("date").copy()
    d["STOCHRSI_K"], d["STOCHRSI_D"] = talib.STOCHRSI(
        d["close"],
        timeperiod=14,
        fastk_period=5,
        fastd_period=3,
        fastd_matype=0
    )
    results.append(d[["symbol", "date", "STOCHRSI_K", "STOCHRSI_D"]])

stochrsi = pd.concat(results, ignore_index=True)
stochrsi["date"] = stochrsi["date"].dt.strftime("%Y/%#m/%#d")
stochrsi

,symbol,date,STOCHRSI_K,STOCHRSI_D
0,T1509,2015/3/20,NaN,NaN
1,T1509,2015/3/23,NaN,NaN
2,T1509,2015/3/24,NaN,NaN
3,T1509,2015/3/25,NaN,NaN
4,T1509,2015/3/26,NaN,NaN
...,...,...,...,...
7334,T2512,2025/5/13,24.581348,29.262463
7335,T2512,2025/5/14,0.000000,8.193783
7336,T2512,2025/5/15,0.783997,8.455115
7337,T2512,2025/5/16,0.000000,0.261332


In [63]:
# TRIX（三重指数移动平均的1日变动率）：用于平滑趋势并过滤价格噪声，适合识别中长期趋势转折

results = []

for symbol in df["symbol"].unique():
    d = df[df["symbol"] == symbol].sort_values("date").copy()
    d["TRIX"] = talib.TRIX(d["close"], timeperiod=30)
    results.append(d[["symbol", "date", "TRIX"]])

trix = pd.concat(results, ignore_index=True)
trix["date"] = trix["date"].dt.strftime("%Y/%#m/%#d")
trix

,symbol,date,TRIX
0,T1509,2015/3/20,NaN
1,T1509,2015/3/23,NaN
2,T1509,2015/3/24,NaN
3,T1509,2015/3/25,NaN
4,T1509,2015/3/26,NaN
...,...,...,...
7334,T2512,2025/5/13,NaN
7335,T2512,2025/5/14,NaN
7336,T2512,2025/5/15,NaN
7337,T2512,2025/5/16,NaN


In [64]:
# ULTOSC（终极振荡器）：结合不同周期动量，减缓单一周期失真，适用于趋势确认和背离识别

results = []

for symbol in df["symbol"].unique():
    d = df[df["symbol"] == symbol].sort_values("date").copy()
    d["ULTOSC"] = talib.ULTOSC(
        d["high"], d["low"], d["close"],
        timeperiod1=7, timeperiod2=14, timeperiod3=28
    )
    results.append(d[["symbol", "date", "ULTOSC"]])

ultosc = pd.concat(results, ignore_index=True)
ultosc["date"] = ultosc["date"].dt.strftime("%Y/%#m/%#d")
ultosc

,symbol,date,ULTOSC
0,T1509,2015/3/20,NaN
1,T1509,2015/3/23,NaN
2,T1509,2015/3/24,NaN
3,T1509,2015/3/25,NaN
4,T1509,2015/3/26,NaN
...,...,...,...
7334,T2512,2025/5/13,42.714050
7335,T2512,2025/5/14,41.478273
7336,T2512,2025/5/15,42.116528
7337,T2512,2025/5/16,47.822847


In [65]:
# WILLR（威廉指标）：衡量当前收盘价在最近高低区间的位置，数值越接近 -100 表示越超卖，越接近 0 表示越超买

results = []

for symbol in df["symbol"].unique():
    d = df[df["symbol"] == symbol].sort_values("date").copy()
    d["WILLR"] = talib.WILLR(d["high"], d["low"], d["close"], timeperiod=14)
    results.append(d[["symbol", "date", "WILLR"]])

willr = pd.concat(results, ignore_index=True)
willr["date"] = willr["date"].dt.strftime("%Y/%#m/%#d")
willr

,symbol,date,WILLR
0,T1509,2015/3/20,NaN
1,T1509,2015/3/23,NaN
2,T1509,2015/3/24,NaN
3,T1509,2015/3/25,NaN
4,T1509,2015/3/26,NaN
...,...,...,...
7334,T2512,2025/5/13,-55.974843
7335,T2512,2025/5/14,-81.132075
7336,T2512,2025/5/15,-80.503145
7337,T2512,2025/5/16,-78.612717


# TA-LIB: Volume Indicator Functions (量价)

In [66]:
# AD（Chaikin A/D 累积/分布线）：基于价格与成交量的关系衡量资金流向，用于识别买盘/卖盘强弱

results = []

for symbol in df["symbol"].unique():
    d = df[df["symbol"] == symbol].sort_values("date").copy()
    d["AD"] = talib.AD(d["high"], d["low"], d["close"], d["volume"])
    results.append(d[["symbol", "date", "AD"]])

ad = pd.concat(results, ignore_index=True)
ad["date"] = ad["date"].dt.strftime("%Y/%#m/%#d")
ad

,symbol,date,AD
0,T1509,2015/3/20,-708.195122
1,T1509,2015/3/23,-137.489240
2,T1509,2015/3/24,-1157.489240
3,T1509,2015/3/25,-1568.822573
4,T1509,2015/3/26,-2641.822573
...,...,...,...
7334,T2512,2025/5/13,26.036481
7335,T2512,2025/5/14,-501.311345
7336,T2512,2025/5/15,444.550724
7337,T2512,2025/5/16,1676.620492


In [67]:
# ADOSC（Chaikin A/D 振荡器）：基于 A/D 线的快慢期 EMA 差值，用于捕捉资金流入流出的短期变化

results = []

for symbol in df["symbol"].unique():
    d = df[df["symbol"] == symbol].sort_values("date").copy()
    d["ADOSC"] = talib.ADOSC(
        d["high"], d["low"], d["close"], d["volume"],
        fastperiod=3, slowperiod=10
    )
    results.append(d[["symbol", "date", "ADOSC"]])

adosc = pd.concat(results, ignore_index=True)
adosc["date"] = adosc["date"].dt.strftime("%Y/%#m/%#d")
adosc

,symbol,date,ADOSC
0,T1509,2015/3/20,NaN
1,T1509,2015/3/23,NaN
2,T1509,2015/3/24,NaN
3,T1509,2015/3/25,NaN
4,T1509,2015/3/26,NaN
...,...,...,...
7334,T2512,2025/5/13,-264.668566
7335,T2512,2025/5/14,-342.017261
7336,T2512,2025/5/15,-41.611318
7337,T2512,2025/5/16,477.087068


In [68]:
# OBV（能量潮指标）：将成交量与价格趋势结合，反映资金累积效果，用于确认趋势方向

results = []

for symbol in df["symbol"].unique():
    d = df[df["symbol"] == symbol].sort_values("date").copy()
    d["OBV"] = talib.OBV(d["close"], d["volume"])
    results.append(d[["symbol", "date", "OBV"]])

obv = pd.concat(results, ignore_index=True)
obv["date"] = obv["date"].dt.strftime("%Y/%#m/%#d")
obv

,symbol,date,OBV
0,T1509,2015/3/20,3332.0
1,T1509,2015/3/23,1946.0
2,T1509,2015/3/24,566.0
3,T1509,2015/3/25,-668.0
4,T1509,2015/3/26,-1741.0
...,...,...,...
7334,T2512,2025/5/13,3681.0
7335,T2512,2025/5/14,2748.0
7336,T2512,2025/5/15,4858.0
7337,T2512,2025/5/16,3149.0


# TA-LIB Volatility Indicator Functions (波动率)

In [71]:
# ATR（平均真实波动幅度）：衡量市场波动性，数值越大表示不确定性越高，常用于设置止损或识别震荡强度

results = []

for symbol in df["symbol"].unique():
    d = df[df["symbol"] == symbol].sort_values("date").copy()
    d["ATR"] = talib.ATR(d["high"], d["low"], d["close"], timeperiod=14)
    results.append(d[["symbol", "date", "ATR"]])

atr = pd.concat(results, ignore_index=True)
atr["date"] = atr["date"].dt.strftime("%Y/%#m/%#d")
atr

,symbol,date,ATR
0,T1509,2015/3/20,NaN
1,T1509,2015/3/23,NaN
2,T1509,2015/3/24,NaN
3,T1509,2015/3/25,NaN
4,T1509,2015/3/26,NaN
...,...,...,...
7334,T2512,2025/5/13,0.253979
7335,T2512,2025/5/14,0.253694
7336,T2512,2025/5/15,0.245931
7337,T2512,2025/5/16,0.244435


In [72]:
# TRANGE（真实波动幅度）：计算当前高低价与前收盘价的最大差异，用于衡量每日的实际波动性

results = []

for symbol in df["symbol"].unique():
    d = df[df["symbol"] == symbol].sort_values("date").copy()
    d["TRANGE"] = talib.TRANGE(d["high"], d["low"], d["close"])
    results.append(d[["symbol", "date", "TRANGE"]])

trange = pd.concat(results, ignore_index=True)
trange["date"] = trange["date"].dt.strftime("%Y/%#m/%#d")
trange

,symbol,date,TRANGE
0,T1509,2015/3/20,NaN
1,T1509,2015/3/23,0.255
2,T1509,2015/3/24,0.345
3,T1509,2015/3/25,0.270
4,T1509,2015/3/26,0.520
...,...,...,...
7334,T2512,2025/5/13,0.160
7335,T2512,2025/5/14,0.250
7336,T2512,2025/5/15,0.145
7337,T2512,2025/5/16,0.225


# TA LIB: Price Transform Functions

In [73]:
# AVGPRICE（平均价格）：计算每日四个价格的加权平均值，简化价格走势分析

results = []

for symbol in df["symbol"].unique():
    d = df[df["symbol"] == symbol].sort_values("date").copy()
    d["AVGPRICE"] = talib.AVGPRICE(d["open"], d["high"], d["low"], d["close"])
    results.append(d[["symbol", "date", "AVGPRICE"]])

avgprice = pd.concat(results, ignore_index=True)
avgprice["date"] = avgprice["date"].dt.strftime("%Y/%#m/%#d")
avgprice

,symbol,date,AVGPRICE
0,T1509,2015/3/20,97.11875
1,T1509,2015/3/23,97.01250
2,T1509,2015/3/24,97.03750
3,T1509,2015/3/25,96.91500
4,T1509,2015/3/26,96.61625
...,...,...,...
7334,T2512,2025/5/13,108.77875
7335,T2512,2025/5/14,108.70250
7336,T2512,2025/5/15,108.62375
7337,T2512,2025/5/16,108.53750


In [74]:
# MEDPRICE（中位价格）：计算每日最高价与最低价的均值，用于衡量价格中心位置

results = []

for symbol in df["symbol"].unique():
    d = df[df["symbol"] == symbol].sort_values("date").copy()
    d["MEDPRICE"] = talib.MEDPRICE(d["high"], d["low"])
    results.append(d[["symbol", "date", "MEDPRICE"]])

medprice = pd.concat(results, ignore_index=True)
medprice["date"] = medprice["date"].dt.strftime("%Y/%#m/%#d")
medprice

,symbol,date,MEDPRICE
0,T1509,2015/3/20,97.2425
1,T1509,2015/3/23,96.9675
2,T1509,2015/3/24,97.0475
3,T1509,2015/3/25,96.8950
4,T1509,2015/3/26,96.6300
...,...,...,...
7334,T2512,2025/5/13,108.7725
7335,T2512,2025/5/14,108.6950
7336,T2512,2025/5/15,108.6025
7337,T2512,2025/5/16,108.5175


In [75]:
# TYPPRICE（典型价格）：加权反映市场真实成交水平，常用于成交量与价量分析模型

results = []

for symbol in df["symbol"].unique():
    d = df[df["symbol"] == symbol].sort_values("date").copy()
    d["TYPPRICE"] = talib.TYPPRICE(d["high"], d["low"], d["close"])
    results.append(d[["symbol", "date", "TYPPRICE"]])

typprice = pd.concat(results, ignore_index=True)
typprice["date"] = typprice["date"].dt.strftime("%Y/%#m/%#d")
typprice

,symbol,date,TYPPRICE
0,T1509,2015/3/20,97.191667
1,T1509,2015/3/23,96.985000
2,T1509,2015/3/24,97.005000
3,T1509,2015/3/25,96.880000
4,T1509,2015/3/26,96.543333
...,...,...,...
7334,T2512,2025/5/13,108.791667
7335,T2512,2025/5/14,108.673333
7336,T2512,2025/5/15,108.613333
7337,T2512,2025/5/16,108.543333


In [76]:
# WCLPRICE（加权收盘价）：给予收盘价更高权重的价格均值，用于趋势跟踪

results = []

for symbol in df["symbol"].unique():
    d = df[df["symbol"] == symbol].sort_values("date").copy()
    d["WCLPRICE"] = talib.WCLPRICE(d["high"], d["low"], d["close"])
    results.append(d[["symbol", "date", "WCLPRICE"]])

wclprice = pd.concat(results, ignore_index=True)
wclprice["date"] = wclprice["date"].dt.strftime("%Y/%#m/%#d")
wclprice

,symbol,date,WCLPRICE
0,T1509,2015/3/20,97.16625
1,T1509,2015/3/23,96.99375
2,T1509,2015/3/24,96.98375
3,T1509,2015/3/25,96.87250
4,T1509,2015/3/26,96.50000
...,...,...,...
7334,T2512,2025/5/13,108.80125
7335,T2512,2025/5/14,108.66250
7336,T2512,2025/5/15,108.61875
7337,T2512,2025/5/16,108.55625
